In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

# Data pre-processing
Loading data. Splitting csv_data into seen_images and seen_labels

In [19]:
seen_csv_data = pd.read_csv("./nn-assignment/trainset.csv")
#seen data = train data + validation data
seen_labels = []
seen_images = []
print(seen_csv_data.shape)
#converting from csv data to (28x28 pixel) image values
for row_index in range(0,seen_csv_data.shape[0]):
    row = seen_csv_data.iloc[row_index].values
    label = row[0]
    img = np.reshape(row[1:],newshape=(28,28))
    seen_labels.append(label)
    seen_images.append(img)

seen_labels = np.array(seen_labels)
seen_images = np.array(seen_images)
print(seen_labels.shape) #shape 28000,1
#print(seen_labels[0:30]) 
print(seen_images.shape)# shape(28000,28,28)

#converting from csv data to (28x28 pixel) image values
test_csv_data = pd.read_csv("./nn-assignment/testset.csv")
test_images = []
print(test_csv_data.shape)
for row_index in range(0,test_csv_data.shape[0]):
    row = test_csv_data.iloc[row_index].values
    img = np.reshape(row,newshape=(28,28))
    test_images.append(img)

test_images = np.array(test_images)
print(test_images.shape)

(28000, 785)
(28000,)
(28000, 28, 28)
(14000, 784)
(14000, 28, 28)


In [17]:
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [20]:
#get train/validation split
train_percent = 0.75

row_len = len(seen_images[:,1])
train_len = round(train_percent * row_len)

train_index = np.random.choice(row_len,size=train_len, replace=False)
#print(len(train_index),train_index[0:50])
#print(len(seen_labels[seen_labels[train_index]]))
train_images = seen_images[train_index]
train_labels = to_categorical(seen_labels[train_index]) 
print(train_images.shape)
train_images = train_images /255 #normalize image values and reshape
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
print(train_images.shape)
#do the same with test data
test_images = test_images /255
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
print(test_images.shape)

val_index = np.delete(np.array(range(0,len(seen_labels))),train_index)
val_images = seen_images[val_index]
val_labels = to_categorical(seen_labels[val_index])
val_images = val_images.reshape(val_images.shape[0], 28, 28, 1)


print(val_images.shape,train_images.shape)

(21000, 28, 28)
(21000, 28, 28, 1)
(14000, 28, 28, 1)
(7000, 28, 28, 1) (21000, 28, 28, 1)



# Creating the model

In [21]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size= (3,3), strides = (1,1), activation = "relu",input_shape = (28,28,1))) #potentially change activation
model.add(MaxPooling2D(pool_size=(2, 2))) #pool_size=(2, 2), strides=None, padding="valid", data_format=None, **kwargs
model.add(Conv2D(filters = 64,kernel_size = (2,2), strides = (2,2), activation = "relu"))
model.add(Flatten())#flatten to enable dense layers (flatten in to 1D)
model.add(Dense(200,activation="relu"))
model.add(Dense(10,activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])

In [22]:
model.fit(train_images,train_labels,validation_data=(val_images,val_labels),epochs = 10, batch_size=32,verbose=1, shuffle=True)

Epoch 1/10
657/657 [==============================] - 6s 9ms/step - loss: 0.7342 - accuracy: 0.7570 - val_loss: 32.1678 - val_accuracy: 0.9309
Epoch 2/10
657/657 [==============================] - 5s 8ms/step - loss: 0.1506 - accuracy: 0.9508 - val_loss: 33.9108 - val_accuracy: 0.9349
Epoch 3/10
657/657 [==============================] - 5s 8ms/step - loss: 0.0848 - accuracy: 0.9731 - val_loss: 25.4154 - val_accuracy: 0.9557
Epoch 4/10
657/657 [==============================] - 5s 8ms/step - loss: 0.0552 - accuracy: 0.9816 - val_loss: 34.9309 - val_accuracy: 0.9456
Epoch 5/10
657/657 [==============================] - 5s 8ms/step - loss: 0.0323 - accuracy: 0.9906 - val_loss: 34.4594 - val_accuracy: 0.9559
Epoch 6/10
657/657 [==============================] - 5s 8ms/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 39.8340 - val_accuracy: 0.9557
Epoch 7/10
657/657 [==============================] - 5s 8ms/step - loss: 0.0133 - accuracy: 0.9957 - val_loss: 49.5876 - val_accuracy: 0.9496

In [23]:
predicted_labels = model(test_images)
#print(predicted_labels.shape)
predicted_labels= [np.argmax(row, axis=None, out=None) for row in predicted_labels]
output = {"ImageId": list(range(1,len(predicted_labels)+1)), "Label": predicted_labels }
output_df = pd.DataFrame(data=output)
#print(output_df.tail())
output_df.to_csv("predictions.csv", index = False)

In [ ]:
#submitted to a kaggle competition. 